In [2]:

#!/usr/bin/env python3
"""
VT/BNDW (or SPY/BND) total-return Monte Carlo via joint block bootstrap
Clean version: no YAML, no audit metadata, no git integration, no SSL bypass
"""

import os
import time
import datetime
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dataclasses import dataclass

# --- CONFIG (edit here) ---
CONFIG = {
    "data": {
        # REQUIRED: Get an API key https://www.alphavantage.co/support/#api-key
        "api_key": "XTKLMIAOG2UP3QDF",
        # Symbols: change to "SPY" and "BND" if desired
        "equity_symbol": "VT",
        "bond_symbol": "BNDW",
        "proxies": {},       # e.g., {"https": "http://proxy:port", "http": "http://proxy:port"}
        "verify_ssl": True   # keep True for security
    },
    "simulation": {
        "months": 360,       # 30 years
        "n_paths": 5000,     # Monte Carlo paths
        "block_size": 36,    # months per bootstrap block
        "seed": 42
    },
    "contributions": {
        "base_monthly": 1000.0,                  # monthly deposit
        "annual_raise": {"pct": 0.02, "apply_in_month": 1},  # 2% raise each January
        "bonus_months": [
            # Example: December bonus
            # {"month": 12, "amount": 5000.0}
        ]
    },
    "rebalancing": {
        "enabled": True,
        "frequency": "annual",     # monthly|quarterly|annual
        "threshold_band": 0.05,    # only rebalance if drift > 5%
        "method": "total"          # total|new_cash_only
    },
    "strategies": [
        {"name": "All-Equity 100/0", "baseline": [1.0, 0.0], "dynamic": None},
        {"name": "Static 60/40",     "baseline": [0.60, 0.40], "dynamic": None},
        {"name": "Static 40/60",     "baseline": [0.40, 0.60], "dynamic": None},
        # Dynamic example A: derisk at -20% drawdown; add bonds after +100% recovery from trough
        {
            "name": "Dynamic A (60/40 -> 80/20 at -20%, -> 40/60 at +100%)",
            "baseline": [0.60, 0.40],
            "dynamic": {
                "dd_trigger": 0.20,
                "up_trigger_gain": 1.00,
                "down_weights": [0.80, 0.20],
                "up_weights":   [0.40, 0.60]
            }
        }
    ],
    "outputs": {
        "summary_csv": "comparison_summary.csv",
        "pairs_csv": "pairwise_comparison.csv",
        "plot_png": "terminal_wealth_hist.png",
        "plot_title": "Equity/Bond Monte Carlo — Contributions & Rebalancing"
    }
}
# --- END CONFIG ---


# 1) Data download: Alpha Vantage Monthly Adjusted (total-return proxy)
def av_monthly_adjusted(symbol, api_key, proxies=None, verify=True, max_retries=8, backoff=4):
    if not api_key:
        raise RuntimeError("Alpha Vantage API key missing. Set CONFIG['data']['api_key'] or ALPHAVANTAGE_API_KEY env var.")
    url = "https://www.alphavantage.co/query"
    params = {"function": "TIME_SERIES_MONTHLY_ADJUSTED", "symbol": symbol, "apikey": api_key}

    last_status = None
    for attempt in range(max_retries):
        try:
            r = requests.get(url, params=params, proxies=proxies, verify=verify, timeout=30)
            last_status = r.status_code
        except Exception:
            time.sleep(backoff * (attempt + 1))
            continue

        if r.status_code != 200:
            time.sleep(backoff * (attempt + 1))
            continue

        data = r.json()
        # Handle rate limits / info messages
        if any(k in data for k in ("Note", "Information", "Error Message")):
            time.sleep(backoff * (attempt + 1))
            continue

        ts = data.get("Monthly Adjusted Time Series")
        if not ts:
            time.sleep(backoff * (attempt + 1))
            continue

        idx = pd.to_datetime(list(ts.keys()))
        adj_close = pd.Series([float(v["5. adjusted close"]) for v in ts.values()], index=idx).sort_index()
        if len(adj_close) < 24:  # need enough history
            time.sleep(backoff * (attempt + 1))
            continue

        return adj_close

    raise RuntimeError(f"Alpha Vantage download failed for {symbol} after {max_retries} attempts (last HTTP {last_status}).")


# 2) Returns + alignment to month-end
def to_monthly_returns(px: pd.Series) -> pd.Series:
    px_m = px.resample("ME").last().dropna()
    r = px_m.pct_change().dropna().clip(lower=-0.99)  # avoid < -100%
    return r


# 3) Contribution schedule
def make_contrib_vector(months: int, base: float, annual_raise_pct: float = 0.0,
                        apply_in_month: int = 1, bonus_months=None) -> np.ndarray:
    if bonus_months is None:
        bonus_months = []
    contrib = np.zeros(months)
    base_current = base
    for t in range(months):
        cal_m = (t % 12) + 1
        contrib[t] += base_current
        for bm in bonus_months:
            if int(bm.get("month", 0)) == cal_m:
                contrib[t] += float(bm.get("amount", 0.0))
        # raise applies for NEXT months after the apply month
        if cal_m == int(apply_in_month) and annual_raise_pct and t < months - 1:
            base_current *= (1.0 + annual_raise_pct)
    return contrib


# 4) Joint block bootstrap (preserve cross-asset correlation)
def block_bootstrap_joint(r_e: pd.Series, r_b: pd.Series, n_months=360, block=12, seed=None):
    if seed is not None:
        np.random.seed(seed)
    df = pd.DataFrame({"re": r_e, "rb": r_b}).dropna()
    N = len(df)
    if N < block:
        raise ValueError(f"History length {N} < block size {block}.")
    max_start = N - block

    chunks = []
    while sum(map(len, chunks)) < n_months:
        s = np.random.randint(0, max_start + 1)
        chunks.append(df.iloc[s:s + block])
    boot = pd.concat(chunks, axis=0).iloc[:n_months]
    return boot["re"].to_numpy(), boot["rb"].to_numpy()


# 5) Strategy spec
@dataclass
class StrategySpec:
    name: str
    baseline: tuple                  # (w_e, w_b) for NEW contributions
    dd_trigger: float = None         # drawdown threshold from equity peak
    up_trigger_gain: float = None    # gain from trough (multiple)
    down_weights: tuple = None       # weights when drawdown condition met
    up_weights: tuple = None         # weights when recovery condition met


# 6) Rebalancing helpers
def is_rebalance_month(t: int, frequency: str) -> bool:
    if frequency == "monthly":
        return True
    elif frequency == "quarterly":
        return ((t + 1) % 3) == 0
    elif frequency == "annual":
        return ((t + 1) % 12) == 0
    return False


def rebalance(eq_val: float, bd_val: float, target_w: tuple, threshold_band: float):
    total = eq_val + bd_val
    if total <= 0:
        return eq_val, bd_val
    current_w_e = eq_val / total
    drift = abs(current_w_e - target_w[0])
    if threshold_band and drift <= threshold_band:
        return eq_val, bd_val
    return total * target_w[0], total * target_w[1]


# 7) Simulator
def simulate_strategy(r_e_hist: pd.Series, r_b_hist: pd.Series, spec: StrategySpec,
                      months: int, n_paths: int, block: int, seed: int,
                      contrib_vec: np.ndarray, rebalance_cfg: dict):
    results = np.zeros(n_paths)
    is_dynamic = spec.dd_trigger is not None and spec.up_trigger_gain is not None

    for i in range(n_paths):
        re, rb = block_bootstrap_joint(r_e_hist, r_b_hist, n_months=months, block=block, seed=seed + i)

        # Equity price index for triggers
        Pe = np.full(months + 1, 100.0)
        for t in range(months):
            Pe[t + 1] = Pe[t] * (1 + re[t])

        last_peak = 100.0
        last_trough = 100.0
        eq = 0.0
        bd = 0.0

        for t in range(months):
            w_e, w_b = spec.baseline
            if is_dynamic:
                P_now = Pe[t]
                last_peak = max(last_peak, P_now)
                drawdown = P_now / last_peak - 1.0
                if drawdown <= -spec.dd_trigger:
                    last_trough = min(last_trough, P_now)
                cond_down = (P_now <= (1.0 - spec.dd_trigger) * last_peak)
                cond_up   = (P_now >= (1.0 + spec.up_trigger_gain) * last_trough)
                if cond_down and spec.down_weights is not None:
                    w_e, w_b = spec.down_weights
                elif cond_up and spec.up_weights is not None:
                    w_e, w_b = spec.up_weights

            # Add new cash
            eq += contrib_vec[t] * w_e
            bd += contrib_vec[t] * w_b

            # Optional rebalancing BEFORE returns
            if rebalance_cfg.get("enabled", False):
                freq = rebalance_cfg.get("frequency", "annual")
                method = rebalance_cfg.get("method", "total")
                band = float(rebalance_cfg.get("threshold_band", 0.0))
                target_w = (w_e, w_b)
                if is_rebalance_month(t, freq):
                    if method == "total":
                        eq, bd = rebalance(eq, bd, target_w, band)
                    # elif method == "new_cash_only": pass

            # Apply returns
            eq *= (1 + re[t])
            bd *= (1 + rb[t])

        results[i] = eq + bd

    return results


# 8) Summary & pairwise util
def summarize(x: np.ndarray):
    return {
        "mean":   float(np.mean(x)),
        "median": float(np.median(x)),
        "std":    float(np.std(x, ddof=1)),
        "p5":     float(np.percentile(x, 5)),
        "p25":    float(np.percentile(x, 25)),
        "p75":    float(np.percentile(x, 75)),
        "p95":    float(np.percentile(x, 95)),
    }


def parse_strategies_from_config(cfg) -> list[StrategySpec]:
    specs = []
    for s in cfg.get("strategies", []):
        dyn = s.get("dynamic")
        specs.append(StrategySpec(
            name=s["name"],
            baseline=tuple(s["baseline"]),
            dd_trigger=None if dyn is None else dyn.get("dd_trigger"),
            up_trigger_gain=None if dyn is None else dyn.get("up_trigger_gain"),
            down_weights=None if dyn is None else tuple(dyn.get("down_weights")),
            up_weights=None if dyn is None else tuple(dyn.get("up_weights")),
        ))
    return specs


# 9) Main
def main():
    cfg = CONFIG
    data_cfg = cfg["data"]
    sim_cfg = cfg["simulation"]
    contrib_cfg = cfg["contributions"]
    rebal_cfg = cfg["rebalancing"]
    out_cfg = cfg["outputs"]

    # Download data
    print("Downloading monthly adjusted (total-return proxy) from Alpha Vantage...")
    eq_px = av_monthly_adjusted(data_cfg["equity_symbol"], data_cfg["api_key"],
                                proxies=data_cfg["proxies"], verify=data_cfg["verify_ssl"])
    bd_px = av_monthly_adjusted(data_cfg["bond_symbol"], data_cfg["api_key"],
                                proxies=data_cfg["proxies"], verify=data_cfg["verify_ssl"])

    # Returns & align
    eq_r = to_monthly_returns(eq_px)
    bd_r = to_monthly_returns(bd_px)
    df_ret = pd.DataFrame({"eq": eq_r, "bd": bd_r}).dropna()
    eq_r, bd_r = df_ret["eq"], df_ret["bd"]
    print(f"Aligned months: {len(eq_r)} (from {eq_r.index.min().date()} to {eq_r.index.max().date()})")

    # Contributions
    CONTRIB = make_contrib_vector(
        months=int(sim_cfg["months"]),
        base=float(contrib_cfg["base_monthly"]),
        annual_raise_pct=float(contrib_cfg.get("annual_raise", {}).get("pct", 0.0)),
        apply_in_month=int(contrib_cfg.get("annual_raise", {}).get("apply_in_month", 1)),
        bonus_months=contrib_cfg.get("bonus_months", []),
    )

    # Strategies
    strategies = parse_strategies_from_config(cfg)

    # Run simulations
    months = int(sim_cfg["months"])
    n_paths = int(sim_cfg["n_paths"])
    block = int(sim_cfg["block_size"])
    seed = int(sim_cfg["seed"])

    all_results = {}
    summaries = []
    print("\n=== Running simulations ===")
    for spec in strategies:
        print(f"Simulating: {spec.name}")
        res = simulate_strategy(eq_r, bd_r, spec, months, n_paths, block, seed, CONTRIB, rebal_cfg)
        all_results[spec.name] = res
        sm = summarize(res)
        sm["strategy"] = spec.name
        summaries.append(sm)

    # Summary CSV
    df_summary = pd.DataFrame(summaries).set_index("strategy")
    df_summary.to_csv(out_cfg["summary_csv"])
    print("\n=== Strategy Summary ===")
    for name in df_summary.index:
        sm = df_summary.loc[name]
        print(f"\n{name}")
        for k in ["mean", "median", "std", "p5", "p25", "p75", "p95"]:
            print(f"{k:>6}: {sm[k]:,.2f}")

    # Pairwise comparison
    pairs = []
    names = list(all_results.keys())
    for i in range(len(names)):
        for j in range(i + 1, len(names)):
            a, b = names[i], names[j]
            A, B = all_results[a], all_results[b]
            pairs.append({
                "A": a,
                "B": b,
                "prob_B_outperforms_A": float(np.mean(B > A)),
                "avg_B_minus_A": float(np.mean(B - A)),
                "median_B_minus_A": float(np.median(B - A)),
            })
    df_pairs = pd.DataFrame(pairs)
    df_pairs.to_csv(out_cfg["pairs_csv"], index=False)
    print("\n=== Pairwise outperformance (B vs A) ===")
    print(df_pairs.to_string(index=False, float_format=lambda x: f"{x:,.4f}"))

    # Plot
    plt.figure(figsize=(11, 7))
    colors = ["#1f77b4","#2ca02c","#ff7f0e","#9467bd","#8c564b","#e377c2","#7f7f7f","#bcbd22","#17becf"]
    for (name, res), c in zip(all_results.items(), colors * ((len(all_results)//len(colors))+1)):
        plt.hist(res, bins=50, alpha=0.45, label=name, color=c)
    plt.xlabel("Terminal Wealth after horizon (USD)")
    plt.ylabel("Frequency")
    title = out_cfg.get("plot_title", "Equity/Bond Monte Carlo")
    plt.title(title)
    plt.legend(fontsize=8, ncol=2)
    plt.tight_layout()
    plt.savefig(out_cfg["plot_png"], dpi=160)

    print("\nWrote files:")
    print(f" - {out_cfg['summary_csv']}")
    print(f" - {out_cfg['pairs_csv']}")
    print(f" - {out_cfg['plot_png']}")


if __name__ == "__main__":
    main()


RuntimeError: Alpha Vantage download failed for VT after 8 attempts (last HTTP None).